In [1]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.9 MB/s eta 0:00:00


In [2]:
!pip install dicom2nifti

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 674.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 35.5 MB/s eta 0:00:00


In [3]:
!pip install nilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 37.3 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pydicom
import dicom2nifti

In [11]:
class Paciente():
  def __init__(self):
    self.__nombre = None
    self.__IdPaciente = 0
    self.listaEstudios = []
    self.dicom = None
    self.nifti= None
    self.dicom = None
    self.anonimizado= None


  def verNombre(self):
    return self.__nombre

  def asignarNombre(self, nombre):
    self.__nombre = nombre

  def verIdPaciente(self):
    return self.__IdPaciente

  def asignarIdPaciente(self, IdPaciente):
    self.__IdPaciente = IdPaciente

  def agregarEstudio(self, estudio):
    self.listaEstudios = self.listaEstudios.append(estudio)

  def verestudio(self,indice):
    self.listaEstudios=self.listaEstudios[indice]

class Estudio():
  def __init__(self):
    self.lista=[]

  def asignarFecha(self,f):
    self.__fecha=f

  def asignarIDestudio(self,I):
    self.__IDe=I

  def asignarIDpaciente(self,Ip):
    self.__IDp=Ip


  def cargarArchivo(self,directorio):
# Iterar sobre todos los archivos en el directorio
    for filename in os.listdir(directorio):

    # Comprobar si el archivo es un archivo DICOM
      if filename.endswith('.dcm'):
          # Construir la ruta completa del archivo
          ruta_completa = os.path.join(directorio, filename)
          # Leer el archivo DICOM y agregarlo a la lista
          dcm_data = pydicom.dcmread(ruta_completa)
          self.lista.append((filename, dcm_data))
          return self.lista


  def convertir(self,rutaDicom, rutaNifti):
    self.dicom = rutaDicom
    self.nifti= rutaNifti
    self.conversion = dicom2nifti.convert_directory(self.dicom, self.nifti)
    print('¡Felicidades! Se ha convertido con éxito')


  def anonimizar(self, ruta, ruta2):
    self.dicom = ruta
    self.anonimizado= ruta2
    datos = ['PatientName', 'PatientBirthDate', 'PatientSex']
    # Iterar sobre todos los archivos en el directorio
    for filename in os.listdir(self.dicom):
        # Comprobar si el archivo es un archivo DICOM
        if filename.endswith('.dcm'):
            # Construir la ruta completa del archivo
            ruta_completa = os.path.join(self.dicom, filename)

            # Leer el archivo DICOM
            dcm_data_anonymous = pydicom.dcmread(ruta_completa)

            # Anonimizar los datos sensibles
            for campo in datos:
                if campo in dcm_data_anonymous:
                    dcm_data_anonymous[campo].value = 'Anonymous'
                elif campo == 'PatientSex':
                    dcm_data_anonymous[campo].value = 'NAN'

            # Construir la ruta completa del archivo anonimizado
            ruta_anonimizada = os.path.join(self.anonimizado, filename)

            # Crear el directorio de destino si no existe
            os.makedirs(self.anonimizado, exist_ok=True)

            # Guardar el archivo DICOM anonimizado
            dcm_data_anonymous.save_as(ruta_anonimizada)

In [18]:
import os
paciente1= Paciente()
paciente1.asignarNombre('Juan')
paciente1.verNombre()

estudio1=Estudio()
estudio=estudio1.cargarArchivo(r'/content/drive/MyDrive/6to semestre/info ll/Trabajo unidad 3/Archivos/0001')
estudio1.convertir('/content/drive/MyDrive/6to semestre/info ll/Trabajo unidad 3/Archivos/0001','/content/drive/MyDrive/6to semestre/info ll/Trabajo unidad 3/Archivos/0001/anonimizados (1)')

¡Felicidades! Se ha convertido con éxito


In [19]:
estudio1.anonimizar('/content/drive/MyDrive/6to semestre/info ll/Trabajo unidad 3/Archivos/0001','/content/drive/MyDrive/6to semestre/info ll/Trabajo unidad 3/Archivos/0001/anonimizados (1)')

In [20]:
paciente1.agregarEstudio(estudio1)

paciente1.asignarIdPaciente(1111)

In [21]:
class Sistema():
    def __init__(self):
        self.pacientes = {}

    def agregar(self, paciente):
        idpaciente = paciente.verIdPaciente()
        self.pacientes[idpaciente] = paciente

    def ingresarEstudio(self,E):
        self.estudio = E

    def ingresarPaciente(self,P):
        self.paciente = P

    def consultarPacientes(self, idPaciente=None):
        if idPaciente: #si existe obtendrá su info
            paciente = self.pacientes.get(idPaciente) # obtiene el paciente del dicc
            if paciente:
                print(f"ID: {idPaciente}, Nombre: {paciente.verNombre()}, Número de estudios: {count(paciente.listaEstudios)}")
            else:
                print(f"No se encontró ningún paciente con ID: {idPaciente}")
        else:
            for idpaciente, paciente in self.pacientes.items(): #iterar sobre los pacientes
                print(f"ID: {idpaciente}, Nombre: {paciente.verNombre()}, Número de estudios: {len(paciente.listaEstudios)}")

    def obtenerDetallesEstudio(self, idPaciente, indiceEstudio):
        paciente = self.pacientes.get(idPaciente) #obtener paciente del dic
        if paciente and 0 <= indiceEstudio < len(paciente.listaEstudios): #clave para un orden esp
            estudio = paciente.listaEstudios[indiceEstudio]
            detalles = []
            for _, dcm_data in estudio.lista:
                detalles.append({
                    "Fecha": dcm_data.StudyDate,
                    "Modalidad": dcm_data.Modality,
                    "Descripción": dcm_data.StudyDescription,
                    "Dimensiones": dcm_data.pixel_array.shape if 'pixel_array' in dcm_data else None
                })
            return detalles
        return None

        # Método para visualizar una imagen NIfTI
    def visualizarImagen(self, rutaNifti):
        import matplotlib.pyplot as plt          # Importa la biblioteca matplotlib para visualización de gráficos
        from nilearn import plotting              # Importa la función plotting de nilearn para visualizar imágenes médicas
        plotting.plot_img(rutaNifti)                # Usa plotting para mostrar la imagen NIfTI
        plt.show()                                         # Muestra la imagen en una ventana gráfica

    def eliminarEstudio(self, idPaciente, indiceEstudio):
        paciente = self.pacientes.get(idPaciente)
        if paciente and 0 <= indiceEstudio < len(paciente.listaEstudios):
            del paciente.listaEstudios[indiceEstudio]
            return True #si existe
        return False #si no existe

    def verificarExiste(self,u):
        return u in self.pacientes

In [17]:
def main():
  sis = Sistema()

  while True:


    print("Bienvenido  ")
    print("Seleccione su solicitud: ")


    a=int(input("\n1.Validar ID del estudio.\n2.Agregar estudio a un paciente.\n3.Consultar un estudio.\n4.Eliminar estudio.\n5.Salir del sistema. \nRespuesta: "))

    if a == 1:
      id_estudio=input("Ingrese el ID del estudio: ")
      if sis.verificarExiste(id_estudio)==True:
        print("\nEste paciente ya se encuentra ingresada en el sistema")
        continue
      else:
          fecha=input("\nIngrese la fecha: ")
          IDestudio=input("\nIngrese el ID del estudio que desea registrar: ")
          et=Estudio()
          et.asignarFecha(fecha)
          et.asignarIDestudio(id_estudio)
          sis.ingresarEstudio(et)

    if a == 2:
      id_estudio=input("\nIngrese el ID del estudio: ")
      if sis.verificarExiste(id_estudio)==True:
        print("\nEste estudio ya se encuentra asignado en el sistema")
        continue
      else:

          id_estudio=input("\nRegistre el ID del estudio: ")
          et=Estudio()

          et.asignarIDpaciente(id_estudio)
          sis.ingresarPaciente(et)

    if a == 3:
        print("Para poder visualizar la información, es necesario ingresar el ID del estudio")
        id_estudio=input("\nIngrese el ID: ")
        if sis.verificarExiste(id_estudio)==False:
          print("\nNo se ha registrado ningun reporte con ese ID\n")
          break

    if a == 4:
            idPaciente = int(input("Ingrese el ID del paciente: "))
            indiceEstudio = int(input("Ingrese el índice del estudio: "))
            if sis.eliminarEstudio(idPaciente, indiceEstudio):
                print("Estudio eliminado exitosamente")
            else:
                print("No se pudo eliminar el estudio")

    if a ==5:
      print("\nHas salido del sistema")
      break

    else:
        print("\nNo ha escogido ninguna opción\n")

if __name__=='__main__':
   main()

Bienvenido  
Seleccione su solicitud: 

1.Validar ID del estudio.
2.Agregar estudio a un paciente.
3.Consultar un estudio.
4.Eliminar estudio.
5.Salir del sistema. 
Respuesta: 5

Has salido del sistema
